In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, ttest_ind, mannwhitneyu
from scipy.stats import pearsonr, spearmanr
from DataLoaders.SubjectLoader import SubjectLoader
from DataLoaders.GutMBLoader import GutMBLoader
from DataLoaders.MBLoader import MBLoader
import seaborn as sns
    
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score
from pingouin import partial_corr
from DataLoaders.CGMLoader import CGMLoader
from scipy.stats import rankdata
import os

from DataLoaders.StudySpecificLoaders.IBDLoader import IBDLoader

from skbio.diversity.alpha import shannon
def do_correl_pearson(ser1, ser2, name1 = 'microbiome column', name2 = 'phenotype column'):
        fig = plt.figure(figsize = (5,5))
        ax = fig.add_subplot(111)MicrobiomeReportFeaturesTenk
        joined = pd.concat([ser1,ser2], axis = 1).dropna()
        ax.plot(joined[ser1.name], joined[ser2.name], 'o', color = 'DarkGrey', markersize = 10, mew = 0, alpha = 0.7)
        ax.set_xlabel(name1)
        ax.set_ylabel(name2)
        
        print(pearsonr(joined[ser1.name], joined[ser2.name]))
        print(spearmanr(joined[ser1.name], joined[ser2.name]))
        ax.grid()
        name_fig = name1.replace(' ', '_').replace('.', '_')+'_'+name2.replace(' ', '_').replace('.', '_')
        print(name_fig)
        plt.savefig('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/microbiome/'+name_fig)
        
def do_pearson(ser1, ser2, name1 = 'microbiome column', name2 = 'phenotype column'):
        joined = pd.concat([ser1,ser2], axis = 1).dropna()
        return pearsonr(joined[ser1.name], joined[ser2.name])
    
def do_mw(ser1, ser2, name1 = 'microbiome column', name2 = 'phenotype column'):
        return mannwhitneyu(ser1[ser1.index.isin(ser2[ser2==True].index)], ser1[ser1.index.isin(ser2[ser2==False].index)])


    
def do_mw(ser1, ser2, name1 = 'microbiome column', name2 = 'phenotype column'):
        return mannwhitneyu(ser1.values, ser2.values)

    
def do_mw_boxplot(ser1, ser2, name1 = 'microbiome column', name2 = 'phenotype column'):
    fig, ax = plt.subplots()
    joined = pd.concat([ser1,ser2], axis = 1).dropna()
    joined.boxplot(ser1.name, by = ser2.name, ax = ax)
   # ax.set_title('')
    plt.show()
    
    
def do_mw_boxplot_2samp(ser1, ser2, name1 = 'microbiome column', name2 = 'phenotype column'):
    fig = plt.figure(figsize = (5,5))
    ax = fig.add_subplot(111)
    ser1.name = 'abund'
    ser1 = pd.DataFrame(ser1)
    ser2.name = 'abund'
    ser2 = pd.DataFrame(ser2)
    ser1['is_t1d'] = 1
    ser2['is_t1d'] = 0
    joined = ser1.append(ser2)
    joined.boxplot('abund', by = 'is_t1d', ax = ax)
   # ax.set_title('')
    plt.show()
        
global_path='/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/microbiome/'


def shannon_divercity(ser):
    return shannon(ser[ser>0.0001].values)

def richness(ser):
    return ser[ser>0.0001].shape[0]

def fdr(p_vals):

    ranked_p_values = rankdata(p_vals)
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1

    return fdr
def do_pearson(ser1, ser2, name1='microbiome column', name2='phenotype column'):
    joined = pd.concat([ser1, ser2], axis=1).dropna()
    return pearsonr(joined[ser1.name], joined[ser2.name]) 

def do_spearman(ser1, ser2, name1='microbiome column', name2='phenotype column'):
    joined = pd.concat([ser1, ser2], axis=1).dropna()
    return spearmanr(joined[ser1.name], joined[ser2.name]) 

def do_correl_pearson_numb_adj(ser1, ser2, covar):
        joined = pd.concat([ser1,ser2, covar], axis = 1).dropna()
        return  partial_corr(joined, ser1.name,ser2.name, covar.name)[['r', 'p-val']].values[0]
    
    
def rename_bact_name(col):
    return col
def compare_2_groups(df, group1, group2, group1_name='main', group2_name='control'):
    comparison_table = pd.DataFrame(columns = [group1_name+' mean', group1_name+' std',\
                                               group2_name+' mean', group2_name+' std',
                                              'ttest_pvalue'])
    signif_cols = []
    comparison_table.loc['number of samples', group1_name+' mean'] = len(set(group1))
    comparison_table.loc['number of samples', group2_name+' mean'] = len(set(group2))
    for col in df.columns:
        A=df[col].loc[group1].dropna().values
        B=df[col].loc[group2].dropna().values
        if len(A)<20:
            continue
        comparison_table.loc[col, group1_name+' mean'] = df[col].loc[group1].dropna().mean()
        comparison_table.loc[col, group1_name+' std'] =  df[col].loc[group1].dropna().std()
        comparison_table.loc[col, group2_name+' mean'] = df[col].loc[group2].dropna().mean()
        comparison_table.loc[col, group2_name+' std'] =  df[col].loc[group2].dropna().std()
        comparison_table.loc[col, 'ttest_pvalue'] =  ttest_ind(A, B)[1]
        comparison_table.loc[col, 'mw_pvalue'] =  mannwhitneyu(A, B)[1]
        if ttest_ind(A, B)[1]<0.05:
         #   print(col)
            signif_cols = signif_cols+[col]
          #  fig, ax = plt.subplots()
          #  draw_plot(A, B, col)
        #    plt.close()
    return comparison_table, signif_cols

def do_a_heatmap_and_corr_matrix(A, B, title=''):
    A = A.rename(columns = {col:rename_bact_name(col) for col in A.columns})
    A = A[[col for col in A.columns if 'unknown' not in col]]
    res_pval = pd.DataFrame()
    res_corr = pd.DataFrame()

            
            
    for col1 in A.columns:
        for col2 in B.columns:
            if col1==col2 or col2=='age' or col1=='age':
                continue
            res_pval.loc[col1, col2] = do_pearson(A[col1], B[col2])[1]
            res_corr.loc[col1, col2] = do_pearson(A[col1], B[col2])[0]


    sign_corr = pd.DataFrame()
    sign_pval = pd.DataFrame()

    for col in res_pval.columns:
        new_pval = fdr(res_pval[col]).sort_values()
        signif = new_pval[new_pval < 0.1]
        if signif.shape[0] > 0:
            for ind in signif.index:
                print(ind, signif.loc[ind])
                do_correl_pearson(A[ind], B[col], ind,  col)
                sign_corr.loc[ind, col] = res_corr.loc[ind, col]
                sign_pval.loc[ind, col] = res_pval.loc[ind, col]
    sign_pval.to_csv(os.path.join(global_path, 'pvalues_'+title+'.csv' ))
    sign_corr.to_csv(os.path.join(global_path,'corr_' + title + '.csv'))
    fig, ax = plt.subplots(figsize = (7, 9))
    cmap = sns.diverging_palette(230,0, 90, 60, as_cmap=True)
# plot heatmap
    sns.heatmap(sign_corr.fillna(0).T, annot=False, fmt=".2f", 
           linewidths=0, cmap=cmap, vmin=-1, vmax=1, square=True, cbar=False)
   # plt.show()
    fig.savefig(os.path.join(global_path, 'sign_corr_'+title+'.png' )
 #   fig.savefig('/net/mraid08/export/genie/LabData/Data/StudySpecificData/tenk_itamar//sign_corr_'+title+'.png' )

In [ ]:
import xgboost
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score,cross_val_predict
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats.stats import pearsonr
import shap

def do_regression(X, y, y_feature_name = '', plot_name = ''):
    model = xgboost.XGBRegressor(learning_rate = 0.005, n_estimators = 2000, reg_lambda = 10, subsample = 0.75, \
                                        colsample_bytree = 0.4, max_depth = 60, reg_alpha = 10, min_child_weight = 50, nthread = 4, seed = 5)

    X1 = X[X.index.isin(y.index)].sort_index()
    y1 = y[y.index.isin(X1.index)].sort_index()

    bins = np.linspace(0, y1.shape[0], 20)


    y_binned = np.digitize(y1, bins)
    kfold = StratifiedKFold(n_splits=5, random_state=17, shuffle=True)
    results = cross_val_predict(model, X1, y1, cv=kfold.split(X1, y_binned))
    matplotlib.rcParams.update({'font.size': 12})
    pred_real = pd.DataFrame(columns = ['y_test', 'y_pred'])
    pred_real['y_test']=y1
    i=0
    for ind in y1.index:
        pred_real.loc[ind, 'y_pred']=results[i]
        i = i+1

    fig = plt.figure(figsize = (5,5))            
    ax = fig.add_subplot(111)
    pred_real = pred_real.rename(columns  ={'y_pred':'Predicted '+y_feature_name, 'y_test':'Measured '+y_feature_name})

    sns.regplot(x=" Predicted ", y=" Measured", data=pred_real.astype(float),x_jitter=.05, ax=ax)

    plt.show()

In [ ]:
t1d_adults_reg_codes = [] ##########load reg codes

control_d2_reg_codes = []  ##########load reg codes

In [ ]:
dates = pd.read_excel('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//enrollment_date.xlsx')


dates['RegistrationCode'] = dates['RegistrationCode'].astype(str)

dates=dates.set_index('RegistrationCode')

dates = dates[dates.index.isin(t1d_adults_reg_codes)]
dates

In [ ]:


all_mbl = GutMBLoader()
segata_species = all_mbl.get_data(df = 'segata_species', reg_ids = t1d_adults_reg_codes, groupby_reg='largest').df

#, study_ids=[6], groupby_reg='largest', take_log = True, min_col_present_frac=0.25).df_metadata.set_index('RegistrationCode')
#[col for col in segata_species.columns if 'uminococcus' in col and 'bromii' in col]

#Ruminococcus bromii

In [ ]:
all_mbl.get_data(df = 'segata_species', reg_ids = t1d_adults_reg_codes, groupby_reg='largest').df_metadata['age'].describe()

In [ ]:
celiac = ['2103540',  '2130360', '2311735', '2718102', '2264805', '2696293', '2768372']

all_mbl = GutMBLoader()
all_mb_md = all_mbl.get_data(df = 'segata_species',reg_ids=t1d_adults_reg_codes, study_ids=[6], groupby_reg='largest', take_log = True, min_col_present_frac=0.25).df_metadata.set_index('RegistrationCode')
#all_mb_md = all_mb_md[~all_mb_md.index.isin(['2565375FD3314_v0_fullrun'])]



td_microbiome_species = all_mbl.get_data(df = 'segata_species',reg_ids=t1d_adults_reg_codes,groupby_reg='first', study_ids=[6],min_col_present_frac=0.35,take_log = True)
td_microbiome_species = td_microbiome_species.df.join(td_microbiome_species.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')

td_microbiome_species = td_microbiome_species[~td_microbiome_species.index.isin(celiac)]


td_microbiome_genus = all_mbl.get_data(df = 'segata_genus',reg_ids=t1d_adults_reg_codes, study_ids=[6], groupby_reg='largest', take_log = True, min_col_present_frac=0.35)
td_microbiome_genus = td_microbiome_genus.df.join(td_microbiome_genus.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')
td_microbiome_genus = td_microbiome_genus[~td_microbiome_genus.index.isin(celiac)]

td_microbiome_family = all_mbl.get_data(df = 'segata_family',reg_ids=t1d_adults_reg_codes, study_ids=[6], groupby_reg='largest', take_log = True, min_col_present_frac=0.35)
td_microbiome_family = td_microbiome_family.df.join(td_microbiome_family.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')

td_microbiome_family = td_microbiome_family[~td_microbiome_family.index.isin(celiac)]
td_microbiome_family = td_microbiome_family.loc[t1d_adults_reg_codes]
print(td_microbiome_family.shape)

In [ ]:
all_mbl = GutMBLoader()
pnp1_microbiome_species = all_mbl.get_data(df = 'segata_species',groupby_reg='first', reg_ids = control_d2_reg_codes, take_log = True,study_ids=[1000],min_col_present_frac=0.3)
pnp1_microbiome_species = pnp1_microbiome_species.df.join(pnp1_microbiome_species.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')




pnp1_microbiome_genus = all_mbl.get_data(df = 'segata_genus', study_ids=[1000], reg_ids = control_d2_reg_codes, groupby_reg='largest', take_log = True, min_col_present_frac=0.25)
pnp1_microbiome_genus = pnp1_microbiome_genus.df.join(pnp1_microbiome_genus.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')


pnp1_microbiome_family = all_mbl.get_data(df = 'segata_family', study_ids=[1000], reg_ids = control_d2_reg_codes,groupby_reg='largest', take_log = True, min_col_present_frac=0.25)
pnp1_microbiome_family = pnp1_microbiome_family.df.join(pnp1_microbiome_family.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')
pnp1_microbiome_family = pnp1_microbiome_family[~pnp1_microbiome_family.index.isin(celiac)]


In [ ]:
from DataLoaders.BloodTestsLoader import BloodTestsLoader
from DataLoaders.BodyMeasuresLoader import BodyMeasuresLoader


btl = BloodTestsLoader()

btld = btl.get_data(groupby_reg='first', study_ids=[6]).df.reset_index().set_index('RegistrationCode')


btld = btld.groupby(btld.index).first()

ml = BodyMeasuresLoader()

mld = ml.get_data(groupby_reg='first',study_ids=[6]).df.reset_index().set_index('RegistrationCode')

mld = mld.groupby(mld.index).first()


from DataLoaders.CGMLoader import CGMLoader
def time_in_good_range(one):
    return one[one>70][one<180].shape[0]/one.shape[0]

cgml=CGMLoader()
cgmld = cgml.get_data(study_ids = [6]).df

cgm_features  = pd.DataFrame()

cgm_features['average'] = cgmld.groupby(level='RegistrationCode')['GlucoseValue'].mean()
cgm_features['noise'] = cgmld.groupby(level='RegistrationCode')['GlucoseValue'].std()
cgm_features['variability'] = cgmld.groupby(level='RegistrationCode')['GlucoseValue'].std()/cgmld.groupby(level='RegistrationCode')['GlucoseValue'].mean()
cgm_features['ppgr_95p'] = cgmld.groupby(level='RegistrationCode')['PPGR'].quantile(0.95)
cgm_features['ppgr_median'] = cgmld.groupby(level='RegistrationCode')['PPGR'].quantile(0.5)
cgm_features['time_in_good_range'] = cgmld.groupby(level='RegistrationCode')['GlucoseValue'].apply(time_in_good_range)


from DataLoaders import DietLoggingLoader
dll = DietLoggingLoader.DietLoggingLoader()

dlld = dll.get_data(study_ids=6)


log = dll.add_nutrients(dlld.df, nutrient_list=['energy_kcal', 'carbohydrate_g', 'protein_g', 'totallipid_g'])
log = log.reset_index()
log['Day']  = log['Date'].astype(str).str[:10]
log = log.reset_index().groupby('RegistrationCode')[['energy_kcal','carbohydrate_g','protein_g','totallipid_g']].sum()
log = log.reset_index()
log['carb_ratio'] = log['carbohydrate_g']/ log['energy_kcal']*400
log['protein_ratio'] = log['protein_g']/ log['energy_kcal']*400
log['lipid_ratio'] = log['totallipid_g']/ log['energy_kcal']*900

cgm_features = cgm_features.join(log.set_index('RegistrationCode')[['carb_ratio','protein_ratio','lipid_ratio' ]])

all_mb_md['year'] = all_mb_md['Date'].apply(lambda x: x.year)
dates = pd.read_excel('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//enrollment_date.xlsx')


dates['RegistrationCode'] = dates['RegistrationCode'].astype(str)
dates=dates.set_index('RegistrationCode')
#all_mb_md['age'] = dates['year']+all_mb_md['age']-2018
#all_mb_md.loc['2590032', age] = 18


md = pd.concat([all_mb_md[['age', 'gender']], mld[['weight', 'bmi']], btld[[
    'bt__hdl_cholesterol',
       'bt__glucose','bt__triglycerides', 
       'bt__ldl_cholesterol', 'bt__hba1c', 
       'bt__total_cholesterol']],cgm_features ]) 





md = md.groupby(md.index).first()
#.dropna(subset=['bt__hba1c'])

###comment out if you want everyone

#md = md[md['age']>17]
md = md[md.index.isin(td_microbiome_family.index)]
#.dropna(subset=['bt__hba1c'])
reg_ids = md.index
md_with_age = md
#[md.index.isin(td_microbiome_family.index)].dropna(subset=['bt__hba1c']).index

In [ ]:
btld[btld.index.isin(t1d_adults_reg_codes)].describe().dropna(axis=1).to_csv(os.path.join(global_path, 'bt_stats.csv'))

In [ ]:
sl = SubjectLoader()

t1d_subj = sl.get_data(groupby_reg='first', study_ids=6)


all_t2d_subj = t1d_subj.df.reset_index().set_index('RegistrationCode')

dates2 = dates.loc[all_t2d_subj.index]


t2d_all_with_dates = pd.concat([all_t2d_subj, dates['year']], axis=1)


t2d_all_with_dates['age'] = t2d_all_with_dates['year']-t2d_all_with_dates['yob']

#print(t2d_all_with_dates[t2d_all_with_dates['age']>17])
t2d_all_with_dates.loc[reg_ids]['age'].mean()

In [ ]:
td_microbiome_family[td_microbiome_family.index.duplicated()]

In [ ]:
dates.loc['2590032']

In [ ]:
md.loc['2590032', 'age'] = 18

In [ ]:
md[['age','gender','weight','bmi','bt__triglycerides','bt__hba1c','average','time_in_good_range', 'carb_ratio','protein_ratio','lipid_ratio']].describe().to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//cohort_basics.csv')
md[['age','gender','weight','bmi','bt__triglycerides','bt__hba1c','average','time_in_good_range', 'carb_ratio','protein_ratio','lipid_ratio']].describe()

In [ ]:
md.columns

md = md[['bt__hdl_cholesterol', 'bt__triglycerides', 'bt__total_cholesterol', 'bt__hba1c',
        'average', 'time_in_good_range', 'bt__ldl_cholesterol']]

In [ ]:
yod = pd.read_excel('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/year_of_diagnosis.xlsx', engine='openpyxl')
yod['RegistrationCode'] = yod['RegistrationCode'].astype(str)
yod = yod[yod['RegistrationCode'].isin(t1d_adults_reg_codes)]
yod['year_Of_diagnosis'] = 2018-yod['Year']
yod['year_Of_diagnosis'].describe()

In [ ]:
from DataLoaders.BloodTestsLoader import BloodTestsLoader
from DataLoaders.BodyMeasuresLoader import BodyMeasuresLoader


btl = BloodTestsLoader()

btld = btl.get_data(groupby_reg='first',reg_ids = control_d2_reg_codes, study_ids=[1000]).df.reset_index().set_index('RegistrationCode')


btld = btld.groupby(btld.index).first()



md_d2 =  btld[['bt__hdl_cholesterol',
       'bt__glucose',  'bt__triglycerides',
       'bt__ldl_cholesterol', 'bt__hba1c',
       'bt__total_cholesterol']]

md_d2

#reg_ids = md[md.index.isin(td_microbiome_family.index)].dropna(subset=['bt__hba1c']).index

In [ ]:
al_micr_t1d = td_microbiome_species.join(td_microbiome_genus).join(td_microbiome_family)

In [ ]:
al_micr_d2 = pnp1_microbiome_species.join(pnp1_microbiome_genus).join(pnp1_microbiome_family)

In [ ]:
sp1= 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_sp|fSGB__1439|gSGB__3529|sSGB__4715'

In [ ]:
rel_pathways = pd.read_pickle('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/relative_pathways.df')
all_path =rel_pathways.notna().sum()
interesting_pathways = all_path[all_path>100].index
rel_pathways = rel_pathways[interesting_pathways].fillna(0.00001)
td_microbiome_family = all_mbl.get_data(df = 'segata_family', study_ids=[6], groupby_reg='largest', take_log = True, min_col_present_frac=0.25)

rel_pathways = rel_pathways.join(td_microbiome_family.df_metadata[['RegistrationCode

rel_pathways = rel_pathways.dropna(subset=['RegistrationCode']).set_index('RegistrationCode')
rel_pathways = rel_pathways[rel_pathways.index.isin(reg_ids)]
do_a_heatmap_and_corr_matrix(rel_pathways, md, 'associations_pathways_adults_pearson.jpg')

In [ ]:
pathway = 'PWY-7208: superpathway of pyrimidine nucleobases salvage'
phen_col = 'average'
fig,ax=plt.subplots(figsize=(6,6))

do_correl_pearson(rel_pathways[pathway].apply(lambda x: np.log10(x)),md[phen_col], name1=pathway, name2 = 'CGM Glucose Average')

plt.show()


In [ ]:
pathway = 'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Paraprevotella|fSGB__592|gSGB__1340'
phen_col = 'bt__hba1c'
fig,ax=plt.subplots(figsize=(6,6))

#sns.regplot(al_micr_t1d[pathway],md[phen_col] )

do_correl_pearson(al_micr_t1d[pathway], md[phen_col], name2 = 'HbA1C (%)', name1='f__Prevotellaceae|fSGB__592')




In [ ]:
do_a_heatmap_and_corr_matrix(rel_pathways.apply(lambda x: np.log10(x)), md[[col for col in md.columns if 'trig' not in col]], 'associations_pathways_adults_pearson_log')

In [ ]:
pathway_log = rel_pathways.apply(lambda x: np.log10(x))

#pathway_log['PWY-7198: pyrimidine deoxyribonucleotides de novo biosynthesis IV'].hist()
#plt.show()


do_correl_pearson(pathway_log['PWY-5677: succinate fermentation to butanoate'], md['bt__triglycerides'])

In [ ]:
do_correl_pearson(al_micr_t1d[sp1], md['protein_ratio'])

In [ ]:
sp1='k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_sp_CAG_122|fSGB__1422|gSGB__3487|sSGB__4659'

In [ ]:
do_correl_pearson(td_microbiome_species[sp1].apply(lambda x: np.log10(x)), md['average'])

In [ ]:
do_a_heatmap_and_corr_matrix(td_microbiome_species, md, 'associations_species_adults_pearson')

In [ ]:
do_a_heatmap_and_corr_matrix(td_microbiome_genus, md, 'associations_genus_adults_pearson')

In [ ]:
td_microbiome_family = all_mbl.get_data(df = 'segata_family',reg_ids=t1d_adults_reg_codes, study_ids=[6], groupby_reg='largest', take_log = True, min_col_present_frac=0.25)
td_microbiome_family = td_microbiome_family.df.join(td_microbiome_family.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')

td_microbiome_family = td_microbiome_family[~td_microbiome_family.index.isin(celiac)]
td_microbiome_family = td_microbiome_family.loc[t1d_adults_reg_codes]
do_a_heatmap_and_corr_matrix(td_microbiome_family, md, 'associations_family_adults_pearson')

In [ ]:
hbac_cols = ['k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|fSGB__592',
            'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Paraprevotella|fSGB__592|gSGB__1340',
            'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|fSGB__1422',
            'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|fSGB__1422']
            


for col in hbac_cols:
    print(col)
    do_correl_pearson(al_micr_d2[col], md_d2['bt__hba1c'], name1 = col, name2 = 'hba1c')
   # do_correl_pearson(al_micr_pnp1[col], md_pnp1['bt__hba1c'])

In [ ]:

trig_cols = ['k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__no_consensus|fSGB__1421',
             'k__Bacteria|p__Firmicutes|c__no_consensus|o__no_consensus|f__no_consensus|fSGB__1451']
for col in trig_cols:
    print(col)
   # do_correl_pearson(td_microbiome_family[col], md['bt__triglycerides'], name1=col, name2='triglycerides')
    do_correl_pearson(al_micr_d2[col], md_d2['bt__triglycerides'].apply(lambda x: 350 if x>300 else x))


In [ ]:
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15339

In [ ]:
chol_columns = ['k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15339']
for col in chol_columns:
    print(col)
   # do_correl_pearson(td_microbiome_species[col], md['bt__total_cholesterol'], name1=col, name2='total_cholesterol')
    do_correl_pearson(al_micr_d2[col], md_d2['bt__total_cholesterol'])

In [ ]:
md_d2.columns

In [ ]:
do_a_heatmap_and_corr_matrix(combine_by_ncbi(td_microbiome_genus), md, 'associations_genus_adults_pearson_ncbi')

In [ ]:
from LabData.DataUtils.tenk_microbiome_report import MicrobiomeReportFeaturesTenk
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', groupby_reg='largest')
all_mb = all_mb.df.join(all_mb.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')
#set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]
set1 =  all_mb[all_mb.index.isin(td_microbiome_family.index)]

t1d_micr_features = MicrobiomeReportFeaturesTenk(set1).return_aggregated_features()



In [ ]:
do_a_heatmap_and_corr_matrix(t1d_micr_features, md, 'associations_features_adults_pearson')

In [ ]:
control_d2_set = ['22001811620303_v0_fullrun', '22001811624815_v0_d', '22001908602331_v0_fullrun', '22001908606092_v0_fullrun', 'sample_22001811623966_v0_fullrun', '22001904611784_v0_fullrun', '22001811620980_v0_fullrun', '22001811625263_v0_d', '22001904601366_v0_fullrun', '22001904604665_v0_fullrun', '22001908603832_v0_fullrun', '22001908606793_v0_fullrun', '22001908611724_v0_fullrun', '22001908612813_v0_fullrun', '22001908614033_v0_fullrun', 'sample_22001908606102_v0_fullrun', '22001904602877_v0_fullrun', '22001904611222_v0_fullrun', '22001910715224_v0_fullrun', '22001808560026_v0_d', '22001811620492_v0_d', '22001811622761_v0_fullrun', '22001811624022_v0_fullrun', '22001901560891_v0_fullrun', '22001901564392_v0_fullrun', '22001901565263_v0_fullrun', '22001904600571_v0_fullrun', '22001904612392_v0_fullrun', '22001901560227_v0_fullrun', '22001901564387_v0_fullrun', '22001904611657_v0_fullrun', '22001908601637_v0_fullrun', '22001908605007_v0_fullrun', '22001908606869_v0_fullrun', '22001807541046_v0_fullrun', '22001811621637_v0_fullrun', '22001811623933_v0_d', '22001901560597_v0_fullrun', '22001901562055_v0_fullrun', '22001901565309_v0_fullrun', '22001904600681_v0_fullrun', '22001904603238_v0_fullrun', '22001904603929_v0_fullrun', '22001904611847_v0_fullrun', '22001904612932_v0_fullrun', '22001904613662_v0_fullrun', '22001908603456_v0_fullrun', '22001908604130_v0_fullrun', '22001908606445_v0_fullrun', '22001908610765_v0_fullrun', '22001908611706_v0_fullrun', '22001908612065_v0_fullrun', '22001908612144_v0_fullrun', '22001908613305_v0_fullrun', '22001908613496_v0_fullrun', '22001908614158_v0_fullrun', '22001910702306_v0_fullrun', '22001910704606_v0_fullrun', 'sample_22001811623249_v0_fullrun', 'sample_22001901561813_v0_fullrun', 'sample_22001901562094_v0_fullrun', '22001808562827_v0_fullrun', '22001808562866_v0_fullrun', '22001811620733_v0_fullrun', '22001811621106_v0_d', '22001811621838_v0_fullrun', '22001811622376_v0_d', '22001811622462_v0_fullrun', '22001811622492_v0_fullrun', '22001811623256_v0_d', '22001904610821_v0_fullrun', '22001904611264_v0_fullrun', 'sample_22001904604342_v0_fullrun', 'sample_22001908600603_v0_fullrun', 'sample_22001908604845_v0_fullrun', '22001806580438_v0_fullrun', '22001901560193_v0_fullrun', '22001901561117_v0_fullrun', '22001901561311_v0_fullrun', '22001901563665_v0_fullrun', '22001901563890_v0_fullrun', '22001904600820_v0_fullrun', '22001904603817_v0_fullrun', '22001904603960_v0_fullrun', '22001904604072_v0_fullrun', '22001904604539_v0_fullrun', '22001904610319_v0_fullrun', '22001904612434_v0_fullrun', '22001904612637_v0_fullrun', '22001904612840_v0_fullrun', '22001811622387_v0_d', '22001811623708_v0_fullrun', '22001811624230_v0_fullrun', '22001901561619_v0_fullrun', '22001904600472_v0_fullrun', '22001904602617_v0_fullrun', '22001904610307_v0_fullrun', '22001904611650_v0_fullrun', '22001904614418_v0_fullrun', '22001908600997_v0_fullrun', '22001811620037_v0_fullrun', '22001811620330_v0_fullrun', '22001811621578_v0_fullrun', '22001811622073_v0_fullrun', '22001811622918_v0_fullrun', '22001811622923_v0_fullrun', '22001811623327_v0_fullrun', '22001811623478_v0_fullrun', '22001811625221_v0_d', '22001811624737_v0_d', '22001908601539_v0_fullrun', '22001908602537_v0_fullrun', '22001908612664_v0_fullrun', '22001908614653_v0_fullrun', '22001910715030_v0_fullrun', '22001804573155_v0_fullrun', '22001811620283_v0_fullrun', '22001811621042_v0_fullrun', '22001811621104_v0_d', '22001811621126_v0_fullrun', '22001811622284_v0_fullrun', '22001807542048_v0_d', '22001808563073_v0_fullrun', '22001811620089_v0_fullrun', '22001811620242_v0_fullrun', '22001807540846_v0_fullrun', '22001808560174_v0_fullrun', '22001810500201_v0_fullrun', '22001811620119_v0_d', '22001811620422_v0_fullrun', '22001811622172_v0_d', '22001811622629_v0_fullrun', '22001811624075_v0_fullrun', '22001811624824_v0_fullrun', '22001901561554_v0_fullrun', '22001901562201_v0_fullrun', '22001901562437_v0_fullrun', '22001808560712_v0_fullrun', '22001901561604_v0_fullrun', '22001901563041_v0_fullrun', '22001904614046_v0_fullrun', '22001904614126_v0_fullrun', '22001904614845_v0_fullrun', '22001908605604_v0_fullrun', '22001908610064_v0_fullrun', '22001805620384_v0_fullrun', '22001808563110_v0_fullrun', '22001808564027_v0_fullrun', '22001811620775_v0_fullrun', '22001811621310_v0_fullrun', '22001811621530_v0_fullrun', '22001808564045_v0_d', '22001901563654_v0_fullrun', '22001901564961_v0_fullrun', '22001908602945_v0_fullrun', '22001804571742_v0_fullrun', '22001806580283_v0_d', '22001808561540_v0_d', '22001808561851_v0_fullrun', '22001810502134_v0_fullrun', '22001811620979_v0_fullrun', '22001811621296_v0_d', '22001811622036_v0_fullrun', '22001811622224_v0_fullrun', '22001808560242_v0_d', '22001811620446_v0_fullrun', '22001811623169_v0_d', '22001811624749_v0_d', '22001901560079_v0_fullrun', '22001901560137_v0_fullrun', '22001811620810_v0_fullrun', '22001811620838_v0_fullrun', '22001811621337_v0_fullrun', '22001811621675_v0_d', '22001811622217_v0_fullrun', '22001811623831_v0_d', '22001808563752_v0_fullrun', '22001811621679_v0_d', '22001811622826_v0_fullrun', '22001811624374_v0_d', '22001804572506_v0_fullrun', '22001811620846_v0_d', '22001811621774_v0_d', '22001811622924_v0_d', '22001811623014_v0_d', '22001811623530_v0_fullrun', '22001807541949_v0_fullrun', '22001811620152_v0_fullrun', '22001811621901_v0_fullrun', '22001811622856_v0_fullrun', '1711522133_v0_fullrun', '22001806580152_v0_fullrun', '22001808560349_v0_d', '22001808562718_v0_fullrun', '22001811620708_v0_fullrun', '22001811621293_v0_d', '22001808561425_v0_fullrun', '22001808563180_v0_fullrun', '22001811621168_v0_d', '22001811622065_v0_fullrun', '22001807540696_v0_fullrun', '22001808560217_v0_d', '22001808563623_v0_fullrun', '22001808561987_v0_d', '22001808562605_v0_fullrun', '22001808563476_v0_d', '22001811620403_v0_fullrun', '22001811620465_v0_d', '22001808561161_v0_fullrun', '22001811620116_v0_fullrun', '1808612517_v0_fullrun', '22001811620948_v0_fullrun', '22001811622346_v0_fullrun', '22001811622869_v0_d', '22001811623422_v0_fullrun', '22001811623465_v0_d', '22001805621063_v0_d', '22001807540014_v0_d', '22001808562141_v0_fullrun', '22001811623015_v0_fullrun', '1711522955_v0_d', '22001807542913_v0_fullrun', '22001808560301_v0_fullrun', '22001811620061_v0_fullrun', '22001811620130_v0_fullrun', '22001811621069_v0_fullrun', '1711521467_v0_d', '22001803510227_v0_fullrun', '22001806581160_v0_fullrun', '22001808563255_v0_fullrun', '22001805620410_v0_d', '22001808560099_v0_d', '22001808560227_v0_fullrun', '22001811620765_v0_fullrun', '22001811621275_v0_d', '22001811624330_v0_d', '22001804571128_v0_d', '22001808561065_v0_d', '22001811624028_v0_fullrun', '22001811624282_v0_fullrun', '22001806581159_v0_fullrun', '22001807540071_v0_fullrun', '22001807540414_v0_fullrun', '22001807540799_v0_fullrun', '22001807542936_v0_fullrun', '22001808563244_v0_fullrun', '22001804571187_v0_fullrun', '22001804572542_v0_fullrun', '22001807540381_v0_fullrun', '22001808563292_v0_fullrun', '22001804572166_v0_d', '22001806580067_v0_d', '22001807540900_v0_fullrun', '22001811620312_v0_fullrun', '22001811620686_v0_fullrun', '22001811620877_v0_fullrun', '1711520771_v0_fullrun', '22001804571413_v0_fullrun', '22001804573900_v0_fullrun', '22001806581783_v0_fullrun', '22001803510797_v0_fullrun', '22001808560172_v0_fullrun', '22001808561013_v0_fullrun', '22001811621289_v0_d', '22001811621934_v0_d', '22001804572591_v0_fullrun', '22001807540972_v0_fullrun', '22001807542904_v0_d', '22001808561052_v0_fullrun', '22001808562734_v0_fullrun', '22001811620017_v0_d', '22001808560735_v0_fullrun', '22001808563585_v0_fullrun', '22001808563989_v0_fullrun', '22001811620010_v0_fullrun', '22001806580835_v0_d', '22001807500438_v0_d', '22001807543042_v0_d', '22001804573508_v0_fullrun', '22001808561641_v0_fullrun', '1905642166_v0_fullrun', '22001807500843_v0_fullrun', '22001807540141_v0_fullrun', '22001807540365_v0_fullrun', '22001808560819_v0_fullrun', '1711522224_v0_fullrun', '22001806580334_v0_fullrun', '22001808560933_v0_d', '22001811622049_v0_d', '22001811624001_v0_d', '22001805620387_v0_fullrun', '22001807540143_v0_d', '22001811620322_v0_fullrun', '22001811620659_v0_fullrun', '22001811621099_v0_fullrun', '22001811622472_v0_d', '22001806581170_v0_d', '22001807540380_v0_fullrun', '22001808560658_v0_fullrun', '22001808560885_v0_d', '22001804571030_v0_fullrun', '22001804573446_v0_fullrun', '22001808561468_v0_fullrun', '22001808562956_v0_fullrun', '22001811620245_v0_d', '22001806581725_v0_fullrun', '22001808561001_v0_d', '22001808562576_v0_fullrun', '22001811620447_v0_fullrun', '22001811621380_v0_d', '22001811621791_v0_d', '22001807540408_v0_fullrun', '22001808560233_v0_fullrun', '22001811620015_v0_fullrun', '22001811621123_v0_d', '22001806514601_v0_fullrun', '22001806581146_v0_fullrun', '22001808560344_v0_d', '22001806514624_v0_fullrun', '22001811620996_v0_d', '22001808562584_v0_fullrun', '22001811620042_v0_fullrun', '22001811620438_v0_fullrun', '22001807540516_v0_fullrun', '22001811620546_v0_d']

In [ ]:
all_mb.df_metadata[all_mb.df_metadata.index.isin(control_d2_set)]['Date'].sort_values()

In [ ]:
print(len(control_d2_set))

In [ ]:
import os
folder = '/net/mraid08/export/genie/LabData/Data/MBPipeline/NovaSeq/tmp2/UNT'
folder_clalit =  '/net/mraid08/export/genie/LabData/Data/MBPipeline/ClalitD2/tmp2/UNT'

all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[6], )
other_samps = all_mb.df_metadata.index

names = []

        
for fastq_file in other_samps:
    if os.path.exists(os.path.join(folder,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder,fastq_file+'.fastq.gz' )]
    elif os.path.exists(os.path.join(folder_clalit,fastq_file+'.fastq.gz' )):
         names+=[os.path.join(folder_clalit,fastq_file+'.fastq.gz' )]
    else:
        print('missing', fastq_file)
        
    print(len(names))
import pickle
with open('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/sample_pathes_t1d.pkl', 'wb') as f:
    pickle.dump(names, f)

In [ ]:
import os
folder = '/net/mraid08/export/genie/LabData/Data/MBPipeline/NovaSeq/tmp2/UNT'
folder_clalit =  '/net/mraid08/export/genie/LabData/Data/MBPipeline/ClalitD2/tmp2/UNT'

all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[6, 19], )
other_samps = all_mb.df_metadata.index

names = []


for fastq_file in control_d2_set:
    if os.path.exists(os.path.join(folder,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder,fastq_file+'.fastq.gz' )]
    elif os.path.exists(os.path.join(folder_clalit,fastq_file+'.fastq.gz' )):
         names+=[os.path.join(folder_clalit,fastq_file+'.fastq.gz' )]
    else:
        print('missing', fastq_file)
        
for fastq_file in other_samps:
    if os.path.exists(os.path.join(folder,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder,fastq_file+'.fastq.gz' )]
    elif os.path.exists(os.path.join(folder_clalit,fastq_file+'.fastq.gz' )):
         names+=[os.path.join(folder_clalit,fastq_file+'.fastq.gz' )]
    else:
        print('missing', fastq_file)


In [ ]:
import os
folder = '/net/mraid08/export/genie/LabData/Data/MBPipeline/NovaSeq/tmp2/UNT'
folder_clalit =  '/net/mraid08/export/genie/LabData/Data/MBPipeline/ClalitD2/tmp2/UNT'


names = []


for fastq_file in control_d2_set:
    if os.path.exists(os.path.join(folder,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder,fastq_file+'.fastq.gz' )]
    elif os.path.exists(os.path.join(folder_clalit,fastq_file+'.fastq.gz' )):
         names+=[os.path.join(folder_clalit,fastq_file+'.fastq.gz' )]
    else:
        print('missing', fastq_file)


import pickle
with open('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/control_sample_pathes.pkl', 'wb') as f:
    pickle.dump(names, f)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, ttest_ind
from DataLoaders.SubjectLoader import SubjectLoader

from DataLoaders.GutMBLoader import GutMBLoader

from DataLoaders.CGMLoader import CGMLoader

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[1000,6], take_log=True, min_col_present_frac=0.2)
all_mb_md = all_mb.df_metadata.set_index('RegistrationCode')
all_mb_md.head(3)

all_mb_md.loc[all_mb_md.index.isin(dates.index), 'Date'] = dates['Date']
#dates

In [ ]:
pnp1_mbl = GutMBLoader()
pnp1_mb = pnp1_mbl.get_data(df = 'segata_species', study_ids=[1], take_log=True, min_col_present_frac=0.2)
pnp1_mb_md = pnp1_mb.df_metadata.set_index('RegistrationCode')
pnp1_mb_md.head(3)

#pnp1_mb_md.loc[pnp1_mb_md.index.isin(dates.index), 'Date'] = dates['Date']

In [ ]:
#'2020-03-23-D2',

excluded_runs = ['2020-03-08-D2', '2020-04-12-D2','2020-05-04-D2','2020-05-24-D2',\
                 '2020-05-27-D2', '2020-06-01-D2',
                '2020-05-27-D2, 2020-06-01-D2']


In [ ]:
mbl = GutMBLoader()

t1d_mbl = mbl.get_data(df='segata_species', study_ids=6)

print(t1d_mbl.df.shape)

In [ ]:
from DataLoaders.BodyMeasuresLoader import BodyMeasuresLoader

bml = BodyMeasuresLoader()

bmld = bml.get_data(study_ids=6)

bmld.df[bmld.df.index.get_level_values(0).isin(t1d_mbl.df_metadata['RegistrationCode'].values)]

In [ ]:
t1d_reg_codes = t1d_mbl.df_metadata['RegistrationCode'].values
len(t1d_reg_codes)

In [ ]:
all_mb_md = all_mb.df_metadata
#all_mb_md = all_mb_md[~all_mb_md.index.isin(['2565375FD3314_v0_fullrun'])]
all_mb_md = all_mb_md.reset_index().set_index('RegistrationCode')
all_mb_md.loc[all_mb_md.index.isin(dates.index), 'Date'] = dates['Date']

all_mb_md['StorageDT'].dropna()

In [ ]:
t2d_mbmd = all_mb_md.dropna(subset=['StorageDT'])

In [ ]:
dates1 = dates.loc[t2d_mbmd.index]

dates1

In [ ]:
all_t2d_subj = t1d_subj.df.reset_index().set_index('RegistrationCode')

dates2 = dates.loc[all_t2d_subj.index]


t2d_all_with_dates = pd.concat([all_t2d_subj, dates2['year']], axis=1)


t2d_all_with_dates['age'] = t2d_all_with_dates['year']-t2d_all_with_dates['yob']

print(t2d_all_with_dates[t2d_all_with_dates['age']>17].shape)

In [ ]:
t2d_mbmd_with_dates = pd.concat([t2d_mbmd, dates1], axis=1)


t2d_mbmd_with_dates['age'] = dates1['year']-t2d_mbmd_with_dates['yob']

#ages[ages>17].shape
#ages[ages<18].shape

In [ ]:
celiac = ['2103540',  '2130360', '2311735', '2718102', '2264805', '2696293', '2768372']
t1d_adults = t2d_mbmd_with_dates[t2d_mbmd_with_dates['age']>17]
t1d_adults = t1d_adults[~t1d_adults.index.isin(celiac)]



t1d_children = t2d_mbmd_with_dates[t2d_mbmd_with_dates['age']<18]
t1d_adults_reg_codes = t1d_adults.index
t1d_children_reg_codes = t1d_children.index

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[1000,6], take_log=True, min_col_present_frac=0.2)

all_mb_md = all_mb.df_metadata
all_mb_md = all_mb_md.reset_index().set_index('RegistrationCode')

all_mb_md['Date'].sort_values().dropna()

Participant with stool samples metadata

control_adults_samples

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[1000,6], take_log=True, min_col_present_frac=0.2)
all_mb_md = all_mb.df_metadata
#all_mb_md = all_mb_md[~all_mb_md.index.isin(['2565375FD3314_v0_fullrun'])]

all_mb_md = all_mb_md.reset_index().set_index('RegistrationCode')
all_mb_md.loc[all_mb_md.index.isin(dates.index), 'Date'] = dates['Date']
all_mb_md = all_mb_md.reset_index().set_index('SampleName')
#all_mb_md = all_mb_md[~all_mb_md['RunName'].isin(excluded_runs)]


#all_mb_md['age'] =  all_mb_md['Date'].apply(lambda x: x.year)-all_mb_md['yob']


print('initial size of samples ', all_mb_md.shape)

all_mb_md['country'] = all_mb_md['country'].fillna('IL')

print('only israeli samples ', all_mb_md[all_mb_md['country']=='IL'].shape)  
#all_mb_md = all_mb_md[all_mb_md['Nextera'].isna()].append(all_mb.df_metadata[all_mb.df_metadata['Nextera']=='None'])

In [ ]:
from DataLoaders.DiagnosesLoader import DiagnosesLoader

dl = DiagnosesLoader()

all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[1000,6], take_log=True, min_col_present_frac=0.2)
all_mb_md = all_mb.df_metadata
all_mb_md = all_mb_md.reset_index().set_index('RegistrationCode')
all_mb_md.loc[all_mb_md.index.isin(dates.index), 'Date'] = dates['Date']
all_mb_md = all_mb_md.reset_index().set_index('SampleName')
#all_mb_md = all_mb_md[~all_mb_md['RunName'].isin(excluded_runs)]


all_mb_md['age'] =  all_mb_md['Date'].apply(lambda x: x.year)-all_mb_md['yob']


print('initial size of samples ', all_mb_md.shape)


all_mb_md = all_mb_md[all_mb_md['Nextera'].isna()].append(all_mb.df_metadata[all_mb.df_metadata['Nextera']=='None'])
all_mb_md['country'] = all_mb_md['country'].fillna('IL')


print('correct sequencing method ', all_mb_md.shape)

diag = dl.get_data(study_ids=[1000,6]).df

excl_deseases = ['fatty_liver_disease', 'autoimmune_disease', 'cholangitis_or_other_bile-related_disease',\
'hepatitis_or_other_liver_disease', 'metabolic_sydrome', 'ulcerative_colitis', 'diverticulosis',\
'gestational_diabetes','type_2_diabetes', 'cancer', 'inflammatory_bowel_disease', 'pancreatic_disease(s)',\
'celiac_disease',  'irritable_bowel_syndrome', 'crohns_disease','pre_diabetes','type_1_diabetes', 'undetermined_colitis',\
 'hiv', 'morbid_obesity','impaired_glucose_tolerance_or_impaired_fasting_glucose']

allowed=[]
for ind in diag.index.get_level_values(0):
    if diag.loc[ind][excl_deseases].max().dropna().max()==0:
        allowed+=[ind]
        
print('only israeli samples ', all_mb_md[all_mb_md['country']=='IL'].shape)   


    
  #  if age<30:
  #      to_add0 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
  #      all_mb_md['age']==age][all_mb_md['gender']==0].head(4*max(number_of_age,2))   
  #      to_add1 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
 #       all_mb_md['age']==age][all_mb_md['gender']==1].head(2*max(number_of_age,2))  
 #       to_add = to_add0.append(to_add1)
 #   else:
   #     to_add0 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
  #      all_mb_md['age']==age][all_mb_md['gender']==0].head(2*number_of_age)
 #       to_add1 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
 #       all_mb_md['age']==age][all_mb_md['gender']==1].head(2*number_of_age)
#        to_add = to_add0.append(to_add1)
  #-  control_d2_samples = control_d2_samples+list(to_add.index)
    
#remove = all_mb_md.loc[control_d2_samples][all_mb_md['age'].isin([68, 67, 66])][all_mb_md['gender']==0].index
    
#control_d2_samples = [samp for samp in control_d2_samples if samp not in remove]

In [ ]:
md_with_age

In [ ]:
day2_healthy = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000]
control_d2_samples = []

print('only healthy participants ', day2_healthy.shape)  
t1d_adults = md_with_age

for age in set(sorted(t1d_adults['age'].values)):
    number_of_age = t1d_adults[t1d_adults['age']==age].shape[0]
    if age==18:
        to_add1 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
        all_mb_md['age'].isin([18,19,20,22,23])].head(4*number_of_age)
        allowed = [a for a in allowed if a not in to_add1['RegistrationCode'].values]
        control_d2_samples = control_d2_samples+list(to_add1.index)
        print(age, number_of_age, len(list(to_add1.index)))
    elif age==21:
        to_add1 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
        all_mb_md['age'].isin([21,22])].head(4*number_of_age)
        allowed = [a for a in allowed if a not in to_add1['RegistrationCode'].values]
        control_d2_samples = control_d2_samples+list(to_add1.index)
        print(age, number_of_age, len(list(to_add1.index)))
    elif age==19:
        to_add1 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
        all_mb_md['age'].isin([19,20])].head(4*number_of_age)
        allowed = [a for a in allowed if a not in to_add1['RegistrationCode'].values]
        control_d2_samples = control_d2_samples+list(to_add1.index)
        print(age, number_of_age, len(list(to_add1.index)))
    else:
        to_add1 = all_mb_md[all_mb_md['country']=='IL'][all_mb_md['RegistrationCode'].isin(allowed)][all_mb_md['StudyTypeID']==1000][
        all_mb_md['age']==age].head(4*number_of_age)
        control_d2_samples = control_d2_samples+list(to_add1.index)
        print(age, number_of_age, len(list(to_add1.index)))

In [ ]:
len(control_d2_samples)

In [ ]:
len(control_d2_samples)

In [ ]:
len(t1d_adults_reg_codes)

In [ ]:
all_mb_md[all_mb_md['RegistrationCode'].isin(t1d_adults_reg_codes)]['RunName'].value_counts().sort_index()

In [ ]:
all_mb_md[all_mb_md['RegistrationCode'].isin(t1d_adults_reg_codes)]['age'].mean()
#.loc[reg_ids]['age'].mean()

In [ ]:
len(set(t1d_adults_reg_codes))

In [ ]:
from DataLoaders.BloodTestsLoader import BloodTestsLoader
from DataLoaders.BodyMeasuresLoader import BodyMeasuresLoader


btl = BloodTestsLoader()

btld = btl.get_data(groupby_reg='first').df.reset_index().set_index('RegistrationCode')


btld = btld.groupby(btld.index).first()

ml = BodyMeasuresLoader()

mld = ml.get_data(groupby_reg='first').df.reset_index().set_index('RegistrationCode')

mld = mld.groupby(mld.index).first()


control_d2_healthy = all_mb_md[all_mb_md.index.isin(control_d2_samples)]
md = pd.concat([all_mb_md.set_index('RegistrationCode')[['age', 'gender']], mld[['weight', 'bmi']], btld[['bt__hba1c']]]) 

md = md.groupby(md.index).first()

compare_2_groups(md, t1d_adults_reg_codes, control_d2_healthy['RegistrationCode'].values, group1_name='t1d adults', group2_name='control adults')[0].round(2)


In [ ]:
compare_2_groups(md, t1d_adults_reg_codes, control_d2_healthy['RegistrationCode'].values, group1_name='t1d adults', group2_name='control adults')[0].round(2).to_csv(os.path.join(global_path, 'chort_cpmparison.csv'))

In [ ]:
len(control_d2_healthy['RegistrationCode'].values)

In [ ]:
old_t2d = md.loc[control_d2_healthy['RegistrationCode'].values]['age'].sort_values().tail(12)

In [ ]:
compare_2_groups(md, t1d_adults_reg_codes, control_d2_healthy['RegistrationCode'].values, group1_name='t1d adults', group2_name='control adults')[0].round(2)

In [ ]:
from LabData.DataUtils.tenk_microbiome_report import MicrobiomeReportFeaturesTenk
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species')
all_mb = all_mb.df.join(all_mb.df_metadata[['RegistrationCode']]).set_index('RegistrationCode')
#set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]
set1 =  all_mb[all_mb.index.isin(md.index)]

t1d_micr_features = MicrobiomeReportFeaturesTenk(set1).return_aggregated_features()



In [ ]:
t2d_samples  = all_mb_md[all_mb_md['RegistrationCode'].isin(t1d_adults.index)].index
t2d_samples = [s for s in t2d_samples if '2565375FD3314_v0_fullrun' not in s]
print(len(t2d_samples))

In [ ]:
t2d_samples_celiac  = all_mb_md[all_mb_md['RegistrationCode'].isin(['2264805', '2696293', '2768372'])].index
t2d_samples_celiac = [s for s in t2d_samples_celiac if '2565375FD3314_v0_fullrun' not in s]
print(len(t2d_samples_celiac))

In [ ]:
set_celiac = all_mb[all_mb.index.isin(t2d_samples_celiac)]
MicrobiomeReportFeaturesTenk(set_celiac).return_aggregated_features()['propionate_producing'].mean()

In [ ]:
control_micr_features['propionate_producing'].sort_values().hist(bins=10)
plt.show()

In [ ]:
all_mb_md.loc[control_micr_features['propionate_producing'].index]['PostHGFRC'].sort_values()

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species')
all_mb = all_mb.df
all_mb.head(3)
set1 = all_mb[all_mb.index.isin(t2d_samples)]

set2 = all_mb[all_mb.index.isin(control_d2_samples)]



t1d_micr_features = MicrobiomeReportFeaturesTenk(set1).return_aggregated_features(
control_micr_features = MicrobiomeReportFeaturesTenk(set2).return_aggregated_features()


print(compare_2_groups(t1d_micr_features.append(control_micr_features),t1d_micr_features.index,
                       control_micr_features.index))

In [ ]:
comparing = compare_2_groups(t1d_micr_features.append(control_micr_features),t1d_micr_features.index,
                       control_micr_features.index)[0]


comparing = comparing.loc[['firm_to_bact_ratio', 'shannon_divercity',
       'richness', 'propionate_producing', 'accetate_producing', 'akkermania',
       'alipastes', 'roseburia', 'eubacterium', 'butirate_producing',
       'faecalibacterium']]
print(comparing['mw_pvalue'])

fdr(comparing['mw_pvalue'].values)

In [ ]:
import os
folder = '/net/mraid08/export/genie/LabData/Data/MBPipeline/NebNext/tmp2/UNT'
folder1 = '/net/mraid08/export/genie/LabData/Data/MBPipeline/NovaSeq/tmp2/UNT'
folder2 = '/net/mraid08/export/genie/LabData/Data/MBPipeline/NebNext_new/tmp2/UNT'
folder_clalit =  '/net/mraid08/export/genie/LabData/Data/MBPipeline/ClalitD2/tmp2/UNT'


names = []
i=0

for fastq_file in control_d2_samples:
    if os.path.exists(os.path.join(folder,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder,fastq_file+'.fastq.gz' )]
    elif os.path.exists(os.path.join(folder_clalit,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder_clalit,fastq_file+'.fastq.gz' )]
    elif os.path.exists(os.path.join(folder1,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder1,fastq_file+'.fastq.gz' )]
        
    elif os.path.exists(os.path.join(folder2,fastq_file+'.fastq.gz' )):
        names+=[os.path.join(folder2,fastq_file+'.fastq.gz' )]
    else:
      #  print('missing', fastq_file)
        i=i+1
print(i)

import pickle
with open('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/control_sample_pathes.pkl', 'wb') as f:
    pickle.dump(names, f)

In [ ]:
i

In [ ]:
compare_2_groups(t1d_micr_features.append(control_micr_features),t1d_micr_features.index,
                       control_micr_features.index)[0].to_excel('comparing_micr_functions_adults.xlsx')

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species')
all_mb = all_mb.df
all_mb.head(3)
set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]

set2 = all_mb[all_mb.index.isin(control_d2_samples)]

pvalues = pd.DataFrame(index = set1.columns, columns = ['pvalue', 'MWp_fdr_HC_MS', 'man_whitney_direction', 'MS_mean', 'HC_mean'])
#pd.DataFrame(index = ibd_microbiome.columns, columns = ['pvalues', 'pvalues_fdr', 'pearson_corr'])
#.MWp_fdr_HC_MS, Bac, MS_mean, HC_mean

for col in set1:
    if (set1[col].fillna(0.0001)[set1[col].fillna(0.0001)>0.0001].shape[0]+set2[col].fillna(0.0001)[set2[col].fillna(0.0001)>0.0001].shape[0])<15:
        continue
  #  if set2[col].fillna(0.0001)[set2[col].fillna(0.0001)>0.0001].shape[0]<10:
  #      continue
    pvalues.loc[col, 'pvalue'] = do_mw(set1.fillna(0.0001)[col], set2.fillna(0.0001)[col])[1]
    pvalues.loc[col, 'man_whitney_direction'] = set1[col].mean()-set2[col].mean()
    pvalues.loc[col, 'MS_mean'] = set1[col].mean()
    pvalues.loc[col, 'HC_mean'] = set2[col].mean()
    if do_mw(set1.fillna(0.0001)[col], set2.fillna(0.0001)[col])[1]<0.05:
        t1d_ser = set1[col]
        t1d_ser.name = 't1d'
        control_ser = set2[col]
        control_ser.name = 'control'
        
        
pvalues = pvalues.dropna(subset = ['pvalue'])
pvalues['MWp_fdr_HC_MS']=fdr(pvalues['pvalue'])

pvalues.index.name = 'Bac'

pvalues.reset_index().to_excel('/net/mraid08/export/jafar/Microbiome/Analyses/T1D/graphlan/my_files/DB//segata_s.xlsx')

all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_genus')
t1d_adults
all_mb = all_mb.df
all_mb.head(3)
set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]

set2 = all_mb[all_mb.index.isin(control_d2_samples)]

pvalues = pd.DataFrame(index = set1.columns, columns = ['pvalue', 'MWp_fdr_HC_MS', 'man_whitney_direction', 'MS_mean', 'HC_mean'])
#pd.DataFrame(index = ibd_microbiome.columns, columns = ['pvalues', 'pvalues_fdr', 'pearson_corr'])
#.MWp_fdr_HC_MS, Bac, MS_mean, HC_mean

for col in set1:
    if (set1[col].fillna(0.0001)[set1[col].fillna(0.0001)>0.0001].shape[0]+set2[col].fillna(0.0001)[set2[col].fillna(0.0001)>0.0001].shape[0])<15:
        continue
  #  if set2[col].fillna(0.0001)[set2[col].fillna(0.0001)>0.0001].shape[0]<10:
  #      continue
    pvalues.loc[col, 'pvalue'] = do_mw(set1.fillna(0.0001)[col], set2.fillna(0.0001)[col])[1]
    pvalues.loc[col, 'man_whitney_direction'] = set1[col].mean()-set2[col].mean()
    pvalues.loc[col, 'MS_mean'] = set1[col].mean()
    pvalues.loc[col, 'HC_mean'] = set2[col].mean()
    if do_mw(set1.fillna(0.0001)[col], set2.fillna(0.0001)[col])[1]<0.05:
        t1d_ser = set1[col]
        t1d_ser.name = 't1d'
        control_ser = set2[col]
        control_ser.name = 'control'
        
        
pvalues = pvalues.dropna(subset = ['pvalue'])
pvalues['MWp_fdr_HC_MS']=fdr(pvalues['pvalue'])
pvalues.index.name = 'Bac'

pvalues.reset_index().to_excel('/net/mraid08/export/jafar/Microbiome/Analyses/T1D/graphlan/my_files/DB//segata_g.xlsx')

all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_family')
t1d_adults
all_mb = all_mb.df
all_mb.head(3)
set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]

set2 = all_mb[all_mb.index.isin(control_d2_samples)]

pvalues = pd.DataFrame(index = set1.columns, columns = ['pvalue', 'MWp_fdr_HC_MS', 'man_whitney_direction', 'MS_mean', 'HC_mean'])
#pd.DataFrame(index = ibd_microbiome.columns, columns = ['pvalues', 'pvalues_fdr', 'pearson_corr'])
#.MWp_fdr_HC_MS, Bac, MS_mean, HC_mean

for col in set1:
    if (set1[col].fillna(0.0001)[set1[col].fillna(0.0001)>0.0001].shape[0]+set2[col].fillna(0.0001)[set2[col].fillna(0.0001)>0.0001].shape[0])<15:
        continue
  #  if set2[col].fillna(0.0001)[set2[col].fillna(0.0001)>0.0001].shape[0]<10:
  #      continue
    pvalues.loc[col, 'pvalue'] = do_mw(set1.fillna(0.0001)[col], set2.fillna(0.0001)[col])[1]
    pvalues.loc[col, 'man_whitney_direction'] = set1[col].mean()-set2[col].mean()
    pvalues.loc[col, 'MS_mean'] = set1[col].mean()
    pvalues.loc[col, 'HC_mean'] = set2[col].mean()
    if do_mw(set1.fillna(0.0001)[col], set2.fillna(0.0001)[col])[1]<0.05:
        t1d_ser = set1[col]
        t1d_ser.name = 't1d'
        control_ser = set2[col]
        control_ser.name = 'control'
        
        
pvalues = pvalues.dropna(subset = ['pvalue'])
pvalues['MWp_fdr_HC_MS']=fdr(pvalues['pvalue'])

pvalues.index.name = 'Bac'

pvalues.reset_index().to_excel('/net/mraid08/export/jafar/Microbiome/Analyses/T1D/graphlan/my_files/DB//segata_f.xlsx')

In [ ]:
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[6,1000], min_col_present_frac=0.2)
all_mb = all_mb.df
all_mb.head(3)
set1_species = all_mb[all_mb.index.isin(t2d_samples)]

set2_species = all_mb[all_mb.index.isin(control_d2_samples)]

all_mb = all_mbl.get_data(df = 'segata_genus', study_ids=[6,1000], min_col_present_frac=0.2)
all_mb = all_mb.df
all_mb.head(3)
set1_genus = all_mb[all_mb.index.isin(t2d_samples)]

set2_genus = all_mb[all_mb.index.isin(control_d2_samples)]
all_mb = all_mbl.get_data(df = 'segata_family', study_ids=[6,1000], min_col_present_frac=0.2)
all_mb = all_mb.df
all_mb.head(3)
set1_family = all_mb[all_mb.index.isin(t2d_samples)]

set2_family = all_mb[all_mb.index.isin(control_d2_samples)]

set1_all = set1_species.join(set1_genus).join(set1_family)
set1_all['group'] = '111'
set2_all = set2_species.join(set2_genus).join(set2_family)
set2_all['group'] = '222'

set_both_all = set2_all.append(set1_all)
set_both_all = set_both_all.reset_index()
del set_both_all['SampleName']

set_both_all.T.to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//X1_y_1_T_abund.csv', sep='\t')




In [ ]:
len(set(set1_all.index))

In [ ]:
all_species_combined = set1_all.join(all_mbl.get_data(df = 'segata_family', groupby_reg='largest', study_ids=[6], min_col_present_frac=0.2).df_metadata['RegistrationCode']).set_index('RegistrationCode')

In [ ]:
all_species_combined = all_species_combined[[col for col in all_species_combined.columns 
                                            if 'unknown' not in col and 'group' not in col]]

all_species_combined = all_species_combined[all_species_combined.index.isin(reg_ids)]

In [ ]:
all_species_combined_grouped = pd.DataFrame()
for col_new in set([c.split('SGB')[0] for c in all_species_combined.columns]):
    if col_new!='group':
        all_species_combined_grouped[col_new[:-2]] = all_species_combined[[col for col in all_species_combined.columns if col_new in col]].sum(axis=1)
all_species_combined_grouped

In [ ]:
do_a_heatmap_and_corr_matrix(all_species_combined_grouped, md, 'associations_species_adults_spearman_combined')

In [ ]:
set_both_all_known = set_both_all[[col for col in set_both_all.columns if 'unknown' not in col]]

set_both_all_known.T.to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//X1_y_1_T_abund_known.csv', sep='\t')


In [ ]:
set_grouped = pd.DataFrame()
for col_new in set([c.split('SGB')[0] for c in set_both_all_known.columns]):
    if col_new!='group':
        set_grouped[col_new[:-2]] = set_both_all_known[[col for col in set_both_all_known.columns if col_new in col]].sum(axis=1)
set_grouped['group'] =    set_both_all_known['group']   
set_grouped.T.to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//X1_y_1_T_abund_known_grouped.csv', sep='\t')
    

In [ ]:
plt.rcParams['figure.figsize'] = [6, 4]
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[1000,6], take_log=True, min_col_present_frac=0.1)
all_mb = all_mb.df
all_mb.head(3)
set1 = all_mb[all_mb.index.isin(t2d_samples)]

set2 = all_mb[all_mb.index.isin(control_d2_samples)]






from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=104)
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp
import numpy as np

set1['is t1d'] = 1

set2['is t1d'] = 0

X = set1.append(set2).fillna(-4)

y = X.reset_index()['is t1d']

#y = y.

del X['is t1d']

#class_weight = dict({0:1.9, 1:35})
#classifier = RandomForestClassifier()
classifier = GradientBoostingClassifier(n_estimators = 2000,learning_rate=.01,max_depth=6,max_features=1,min_samples_leaf=10)
#classifier = svm.SVC(kernel='linear', probability=True,
#                     random_state=random_state)
#
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
y_test_all = []
y_predict_all = []
y_pred = []
y_true = []
trhresh_all = []

i = 0
for i, (train, test) in enumerate(cv.split(X, y)):
    fitted = classifier.fit(X.iloc[train], y.iloc[train])
    probas_ = fitted.predict_proba(X.iloc[test])
   # y_pred += [fitted.predict(X.iloc[test])]
   # y_true +=[y.iloc[test]]
  #  y_predict_all+=[probas_[:, 1]]
  #  y_test_all+=[y.iloc[test]]
    fpr, tpr, thresholds = roc_curve(y.iloc[test], probas_[:, 1])
    trhresh_all+=[thresholds]
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3)
           #  label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Prediction of T1D from microbiome')
plt.legend(loc="lower right")
plt.savefig('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//predictor_t1d_hc.jpg', dpi=800)

In [ ]:
from sklearn.preprocessing import StandardScaler
import seaborn as sns
set1['is t1d'] = 'T1DM'

set2['is t1d'] = 'Healthy Controls'

X = set1.append(set2).fillna(-4)

y = X.reset_index()['is t1d']

#y = y.

del X['is t1d']

X_scaled = StandardScaler().fit_transform(X)
X_scaled[:5]
features = X_scaled.T
cov_matrix = np.cov(features)
cov_matrix[:5]
values, vectors = np.linalg.eig(cov_matrix)
values[:5]
explained_variances = []
for i in range(len(values)):
    explained_variances.append(values[i] / np.sum(values))
    
    
projected_1 = X_scaled.dot(vectors.T[0])
projected_2 = X_scaled.dot(vectors.T[1])
res = pd.DataFrame(projected_1, columns=['PC1, 7.7%'])
res['PC2, 4.4 %'] = projected_2
res['group'] = y
res.head()


plt.figure(figsize=(12, 10))
sns.scatterplot(res['PC1, 7.7%'], res['PC2, 4.4 %'], hue=res['group'], s=100)

In [ ]:
explained_variances
#*100

In [ ]:
import pandas as pd
import numpy as np
from sklearn import manifold
import scipy.stats
import matplotlib.pyplot as plt
import itertools
from scipy.spatial import distance
from scipy.spatial.distance import braycurtis

def BrayCurtis(X):

    X = np.array(X)
    n_samples = X.shape[0] 
    n_distance = int(n_samples * (n_samples - 1) / 2)
    d_array = np.zeros((n_distance))
    for i, (idx1, idx2) in enumerate(itertools.combinations(range(n_samples),2)):
        d_array[i] = braycurtis(X[idx1], X[idx2])    
    return squareform(d_array)

def executePCoA(data, distance_metric='BrayCurtis', drawBiplot=False, n_arrows=False, groupfile=False):    
    matrix = data.values
    n_features, n_samples = matrix.shape
    print(n_features,'features, ',n_samples,'samples')
    

    if distance_metric == 'BrayCurtis':
        distance_matrix = distance.squareform(distance.pdist(matrix.T, metric="braycurtis"))

 
  # execute PCoA
    mds = manifold.MDS(n_components=2, max_iter=3000, dissimilarity="precomputed", n_jobs=1)
    positions = mds.fit(distance_matrix).embedding_
    positions_with_sampleIndex = pd.DataFrame(positions, index=data.columns)
    
    # General settings of the canvas
    fig = plt.figure(figsize=(12,12))
    ax = fig.gca()
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.spines['bottom'].set_position(('data',0))
    ax.spines['left'].set_position(('data',0))
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
#    ax.set_xlim(-0.02,0.02)
#    ax.set_ylim(-0.02,0.02)
    
  
    # draw plots using colors if samples are binned into groups
    if True:
        group_names = []
        group2sample = {}
        df = groupfile
        for sample in df.index:
            # Use the value of second column (1) as a grouping category.
            group = df.loc[sample]
            if group in group2sample:
                group2sample[group].append(sample)
            else:
                group2sample[group] = [sample]
                group_names.append(group)
        colors = itertools.cycle(['r','g','b','c','m','y','k'])
        markers = itertools.cycle(['o','^','s','*','x'])
        for i,current_group in enumerate(group_names):
            if len(group2sample[current_group]) == 0:
                continue
            ax.scatter(positions_with_sampleIndex.loc[group2sample[current_group],0],
                       positions_with_sampleIndex.loc[group2sample[current_group],1],
                       s=100, marker=next(markers), color=next(colors), label='%s'%current_group)
        plt.legend(bbox_to_anchor=(0., 1.01, 1., 1.01), loc=3, ncol=6, mode="expand", borderaxespad=0.)
    else:
        for i,sample_name in enumerate(data.columns):
            ax.annotate(sample_name, xy=(positions[i,0],positions[i,1]), xytext=(5,5), textcoords='offset points', color='k', fontsize=16)
        ax.scatter(positions[:,0], positions[:,1], c='k', s=50)
    
    x_label = 'PCo1'
    y_label = 'PCo2'
 #   ax.annotate(x_label, xy=(0.0, -1.0), xytext=(0.0,-40.0), textcoords='offset points', ha='center', color='k', fontsize=18)
#    ax.annotate(y_label, xy=(-1.0, 0.0), xytext=(-40.0,0.0), textcoords='offset points', ha='center', color='k', fontsize=18, rotation=90)
    plt.show()

set1['is t1d'] = 'T1DM'

set2['is t1d'] = 'Healthy Controls'

X = set1.append(set2).fillna(0.0001)

y = X['is t1d']

#y = y.

del X['is t1d']


X_t = X[[col for col in X.columns if X[col].quantile(0.7)>0.0002]].T.reset_index().fillna(0.0001)

X_t['sp'] = X_t['index'].apply(lambda x: x.split('|fSGB')[0])

del X_t['index']

X_t = X_t.groupby(['sp']).sum()
#executePCoA(X.apply(lambda x: np.log10(x)).T, groupfile = y)

executePCoA(X_t, groupfile = y)

In [ ]:
X[[col for col in X.columns if X[col].quantile(0.7)>0.0002]].shape

In [ ]:
X_t = X[[col for col in X.columns if X[col].quantile(0.7)>0.0002]].T.reset_index().fillna(0.0001)




X_t['sp'] = X_t['index'].apply(lambda x: x.split('|fSGB')[0])

del X_t['index']

X_t = X_t.groupby(['sp']).sum()
X_t
#

#_t[[col for col in X_t.columns if X_t[col].mean()>0.005]]

In [ ]:
X

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
tsne =TSNE(n_components=2, init='random', perplexity = 100, random_state=0)


X_tsne = tsne.fit_transform(X)

vis_x = X_tsne[:, 0]
vis_y = X_tsne[:, 1]

res['Y'] = y
res.head()
res['tSNE1'] = vis_x
res['tSNE2'] = vis_y

plt.figure(figsize=(12, 10))
sns.scatterplot(res['tSNE1'], res['tSNE2'], hue=res['group'], s=100)

In [ ]:
from DataLoaders import DietLoggingLoader
dll = DietLoggingLoader.DietLoggingLoader()

dlld = dll.get_data(study_ids=[1])


log = dll.add_nutrients(dlld.df, nutrient_list=['energy_kcal',
 'carbohydrate_g',
 'protein_g',
 'totallipid_g',
 'totaldietaryfiber_g',
 'totalsaturatedfattyacids_g']).reset_index()

log = log.groupby('RegistrationCode')[['energy_kcal',
 'carbohydrate_g',
 'protein_g',
 'totallipid_g',]].sum()

log['carb_perc'] = log['carbohydrate_g']/log['energy_kcal']*400
log['proten_perc'] = log['protein_g']/log['energy_kcal']*400
log['totallipid_g'] = log['totallipid_g']/log['energy_kcal']*900

log[['carb_perc', 'proten_perc', 'totallipid_g']].describe()

In [ ]:
def bacteria_name(x):
    if x.split('|')[6].replace('s__', '')!='unknown':
        return x.split('|')[6].replace('s__', '')+'_'+ x.split('|')[9]
    elif x.split('|')[5].replace('g__', '')!='unknown':
        return x.split('|')[5].replace('g__', '')+' '+x.split('|')[9]
    elif x.split('|')[4].replace('f__', '')!='unknown':
        return x.split('|')[4].replace('f__', '')+' '+x.split('|')[9]
    else:
        return x.split('|')[9]
    
    
X = X.rename(columns = {col:bacteria_name(col).replace('unclassified', '') for col in X.columns})

In [ ]:
import shap
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]

fitted = classifier.fit(X, y)



shap_values = shap.TreeExplainer(fitted)

import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

shap_values = shap_values.shap_values(X)
fig = shap.summary_plot(shap_values, X, max_display=15, plot_type="layered_violin", color='coolwarm', show=False)


plt.savefig('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//predictor_t1d_hc_shap.jpg', dpi=800,bbox_inches='tight')
#p

In [ ]:
set_both_all_known[[col for col in set_both_all_known if 'no_consensus' not in col]]

In [ ]:
set_grouped = pd.DataFrame()
set_both_all_known = set_both_all_known[[col for col in set_both_all_known if 'no_consensus' not in col]]
for col_new in set([c.split('SGB')[0] for c in set_both_all_known.columns]):
    if col_new!='group':
        set_grouped[col_new[:-2]] = set_both_all_known[[col for col in set_both_all_known.columns if col_new in col]].sum(axis=1)
set_grouped['group'] =    set_both_all_known['group']   
set_grouped.T.to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//X1_y_1_T_abund_known_grouped.csv', sep='\t')
    

In [ ]:
y.name = 'is_t1d'
X.reset_index().join(y).T.to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//X_y_1_T.csv', sep='\t')

In [ ]:
set1['is t1d'] = 1

set2['is t1d'] = 0

X = set1.append(set2).fillna(-4)
X = X[[col for col in X.columns if 'unknown' not in col]]
y = X.reset_index()['is t1d']

#y = y.

del X['is t1d']
y1  =y.apply(lambda x: 'T1D' if x==1 else 'HC')
X1 = X.reset_index()
del X1['SampleName']
y1.name = 'is_t1d'
X1.join(y1).T.to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//X1_y_1_T_known.csv', sep='\t')

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

#shap_values = shap_values.shap_values(X)
fig = shap.summary_plot(shap_values, X, max_display=15, plot_type="layered_violin", color='coolwarm', show=False)


plt.savefig('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//predictor_t1d_hc_shap.jpg', dpi=800,bbox_inches='tight')
#plt.savefig('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//predictor_t1d_hc_shap.jpg', dpi=800)

In [ ]:
butirate_procucing = ['k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15316',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15318',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15322',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15323',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15332',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15333',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15339',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15342',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__673|gSGB__1693|sSGB__2326',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Butyrivibrio|s__Butyrivibrio_crossotus|fSGB__1475|gSGB__3732|sSGB__5065',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Coprococcus|s__Coprococcus_eutactus|fSGB__1480|gSGB__3759|sSGB__5117',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_ventriosum|fSGB__1472|gSGB__3721|sSGB__5045',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Anaerostipes|s__Anaerostipes_hadrus|fSGB__1413|gSGB__3421|sSGB__4540',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Anaerostipes|s__Anaerostipes_hadrus|fSGB__1413|gSGB__3421|sSGB__4547',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Oscillospiraceae|g__Oscillibacter|s__Oscillibacter_sp|fSGB__3056|gSGB__9624|sSGB__15078',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Oscillospiraceae|g__Oscillibacter|s__Oscillibacter_sp_CAG_241|fSGB__3056|gSGB__9633|sSGB__15090',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_faecis|fSGB__1454|gSGB__3634|sSGB__4925',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Coprococcus|s__unknown|fSGB__1428|gSGB__3495|sSGB__4669',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Coprococcus|s__Coprococcus_catus|fSGB__1428|gSGB__3495|sSGB__4670',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__unknown|s__unknown|fSGB__1445|gSGB__3570|sSGB__4777',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Lachnospiraceae_unclassified|s__Eubacterium_rectale|fSGB__1454|gSGB__3638|sSGB__4933',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_inulinivorans|fSGB__1454|gSGB__3642|sSGB__4940',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_hallii|fSGB__1413|gSGB__3419|sSGB__4532',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_hallii|fSGB__1413|gSGB__3419|sSGB__4537',
 'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_intestinalis|fSGB__1454|gSGB__3644|sSGB__4951']


roseburia = ['k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_sp_CAG_100|fSGB__1451|gSGB__3612|sSGB__4882',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_sp_CAG_182|fSGB__1454|gSGB__3646|sSGB__4953',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_sp_CAG_197|fSGB__1454|gSGB__3633|sSGB__4922',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_sp_CAG_303|fSGB__1424|gSGB__3490|sSGB__4664',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_sp_CAG_380|fSGB__1421|gSGB__3484|sSGB__4654',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_sp_CAG_471|fSGB__1449|gSGB__3604|sSGB__4867',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Roseburia|s__Roseburia_sp|fSGB__1454|gSGB__3641|sSGB__4938']


faecalibacterium = ['k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15316',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15318',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15322',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9740|sSGB__15323',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15332',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15333',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15339',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__3058|gSGB__9741|sSGB__15342',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|fSGB__673|gSGB__1693|sSGB__2326']


eubacterium = ['k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_An11|fSGB__1432|gSGB__3505|sSGB__4681',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_115|fSGB__1325|gSGB__3175|sSGB__4191',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_156|fSGB__1472|gSGB__3720|sSGB__5043',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_161|fSGB__1472|gSGB__3719|sSGB__5042',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_180|fSGB__1349|gSGB__3293|sSGB__4348',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_192|fSGB__1472|gSGB__3723|sSGB__5051',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_202|fSGB__1336|gSGB__3244|sSGB__4290',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_248|fSGB__1476|gSGB__3740|sSGB__5077',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_251|fSGB__1342|gSGB__3277|sSGB__4327',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_252|fSGB__1476|gSGB__3740|sSGB__5076',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_274|fSGB__1380|gSGB__3363|sSGB__4447',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_38|fSGB__1476|gSGB__3746|sSGB__5089',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_581|fSGB__1336|gSGB__3235|sSGB__4272',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_603|fSGB__1472|gSGB__3728|sSGB__5058',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_786|fSGB__1325|gSGB__3176|sSGB__4193',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_841|fSGB__2947|gSGB__9285|sSGB__14238',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_86|fSGB__1476|gSGB__3744|sSGB__5087',
'k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_ventriosum|fSGB__1472|gSGB__3721|sSGB__5045']

alipastes = ['k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_sp_An31A|fSGB__673|gSGB__1677|sSGB__2304',
'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_sp_An66|fSGB__673|gSGB__1678|sSGB__2306',
'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_sp_CAG_268|fSGB__673|gSGB__1681|sSGB__2313',
'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_sp_CAG_435|fSGB__658|gSGB__1627|sSGB__2230',
'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_sp_CAG_514|fSGB__658|gSGB__1627|sSGB__2227',
'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_sp_CHKCI003|fSGB__673|gSGB__1674|sSGB__2292']


akkermania = ['k__Bacteria|p__Verrucomicrobia|c__Verrucomicrobiae|o__Verrucomicrobiales|f__Akkermansiaceae|g__Akkermansia|s__Akkermansia_muciniphila|fSGB__2382|gSGB__6531|sSGB__9226', 'k__Bacteria|p__Verrucomicrobia|c__Verrucomicrobiae|o__Verrucomicrobiales|f__Akkermansiaceae|g__Akkermansia|s__Akkermansia_muciniphila|fSGB__2382|gSGB__6531|sSGB__9228']
#.mean()

all_mbl = GutMBLoader()
all_mb_g = all_mbl.get_data(df = 'segata_species')
all_mb = all_mb_g.df
set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]
set2 = all_mb[all_mb.index.isin(control_d2_samples)]
#loc[butirate_procucing]

do_mw(set1[butirate_procucing].sum(axis=1), set2[butirate_procucing].sum(axis=1))

In [ ]:
do_mw(set1[faecalibacterium].sum(axis=1), set2[faecalibacterium].sum(axis=1))

In [ ]:
colors = [(0.7, 0.15, 0.15, 0.6), (0.5, 0.8, 0.5, 0.8)]

def draw_plot(A, B, title=''):
    fig, ax = plt.subplots(figsize = [3, 4])
    bplot1 = ax.boxplot([A, B], vert=True,  patch_artist=True,  labels=['T1D', 'Healthy Control'], showfliers = False)  # will be used to label x-ticks
    for bplot in (bplot1,):
        for patch, color in zip(bplot['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_linewidth(0.5)
    for median in bplot['medians']:
        median.set(color='k', linewidth=1,)
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(11)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(11)
    ax.set_ylabel('log abundance',  fontsize=11)
    ax.set_title(title,  fontsize=13)
    ax.tick_params(direction='in')
    plt.savefig('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//microbiome//title.jpg', dpi=800,bbox_inches='tight')

    
    
#draw_plot(set1[faecalibacterium].sum(axis=1).apply(lambda x: np.log(x)).values, set2[faecalibacterium].sum(axis=1).apply(lambda x: np.log(x)).values, 'Faecalibacterium prausnitzii abundance')

In [ ]:
pd.concat([X, y])

In [ ]:
y

In [ ]:
t1d_micr_features['propionate_producing'].values

In [ ]:
do_mw(t1d_micr_features['propionate_producing'].apply(lambda x: np.log(x)), control_micr_features['propionate_producing'].apply(lambda x: np.log(x)))

In [ ]:
colors = [(0.7, 0.15, 0.15, 0.6), (0.5, 0.8, 0.5, 0.8)]


    
    
draw_plot(t1d_micr_features['accetate_producing'].apply(lambda x: np.log(x)).values, control_micr_features['accetate_producing'].apply(lambda x: np.log(x)).values, 'Propionate Producing Bacteria')

In [ ]:
colors = [(0.7, 0.15, 0.15, 0.6), (0.5, 0.8, 0.5, 0.8)]

def draw_plot(A, B, title=''):
    fig, ax = plt.subplots(figsize = [3, 4])
    bplot1 = ax.boxplot([A, B], vert=True,  patch_artist=True,  labels=['T1D', 'Healthy Control'], showfliers = False)  # will be used to label x-ticks
    for bplot in (bplot1,):
        for patch, color in zip(bplot['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_linewidth(0.5)
    for median in bplot['medians']:
        median.set(color='k', linewidth=1,)
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(10)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(10)
    ax.set_title(title,  fontsize=12)
    ax.tick_params(direction='in')
    plt.show()
    
    
draw_plot(set1[faecalibacterium].sum(axis=1).values, set2[faecalibacterium].sum(axis=1).values, 'Faecalibacterium prausnitzii abundance')

In [ ]:
print(set1[faecalibacterium].sum(axis=1).mean())
print(set2[faecalibacterium].sum(axis=1).mean())


print(ttest_ind(set1[faecalibacterium].sum(axis=1), set2[faecalibacterium].sum(axis=1)))

print(do_mw(set1[faecalibacterium].sum(axis=1), set2[faecalibacterium].sum(axis=1)))

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[1000,6])
all_mb = all_mb.df
all_mb.head(3)
set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]

set2 = all_mb[all_mb.index.isin(control_d2_samples)]

print(ttest_ind(set1.fillna(0.0001).apply(lambda x: shannon_divercity(x), axis=1), set2.fillna(0.0001).apply(lambda x: shannon_divercity(x), axis=1)))
print(ttest_ind(set1.apply(lambda x: richness(x), axis=1), set2.apply(lambda x: richness(x), axis=1)))

In [ ]:
print(set1.fillna(0.0001).apply(lambda x: shannon_divercity(x), axis=1).mean(), set2.fillna(0.0001).apply(lambda x: shannon_divercity(x), axis=1).mean())
print(set1.apply(lambda x: richness(x), axis=1).mean(), set2.apply(lambda x: richness(x), axis=1).mean())

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', study_ids=[1000,6], take_log=True, min_col_present_frac=0.2)
all_mb = all_mb.df
all_mb.head(3)
set1 = all_mb[all_mb.index.isin(t1d_adults['SampleName'].values)]

set2 = all_mb[all_mb.index.isin(control_d2_samples)]


from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=105)
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from scipy import interp
import numpy as np

set1['is t1d'] = 1

set2['is t1d'] = 0

X = set1.append(set2).fillna(-4)

y = pd.Series(np.random.randint(2, size=X.shape[0]))

#y = y.

del X['is t1d']

#class_weight = dict({0:1.9, 1:35})
#classifier = RandomForestClassifier()
classifier = GradientBoostingClassifier(n_estimators = 1000,learning_rate=.01,max_depth=4,max_features=1,min_samples_leaf=10)
#classifier = svm.SVC(kernel='linear', probability=True,
#                     random_state=random_state)
#
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
y_test_all = []
y_predict_all = []
y_pred = []
y_true = []
trhresh_all = []

i = 0
for i, (train, test) in enumerate(cv.split(X, y)):
    fitted = classifier.fit(X.iloc[train], y.iloc[train])
    probas_ = fitted.predict_proba(X.iloc[test])
   # y_pred += [fitted.predict(X.iloc[test])]
   # y_true +=[y.iloc[test]]
  #  y_predict_all+=[probas_[:, 1]]
  #  y_test_all+=[y.iloc[test]]
    fpr, tpr, thresholds = roc_curve(y.iloc[test], probas_[:, 1])
    trhresh_all+=[thresholds]
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3)
           #  label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Prediction of T1D from microbiome')
plt.legend(loc="lower right")
plt.show()

In [ ]:
ttest_ind(set1.fillna(0.0001).apply(lambda x: shannon_divercity(x), axis=1), set2.fillna(0.0001).apply(lambda x: shannon_divercity(x), axis=1))
ttest_ind(set1.apply(lambda x: richness(x), axis=1), set2.apply(lambda x: richness(x), axis=1))

In [ ]:

t1d_adults_numbers = pd.Series(list(t1d_adults.index))
t1d_adults_numbers.name='RegistrationCode'
t1d_adults.to_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/t1d_adults.csv')

In [ ]:
pd.read_csv('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/t1d_adults.csv')

In [ ]:
list(control_d2_metadata['RegistrationCode'].values)

In [ ]:
list(control_meta.index)

In [ ]:
control_d2_metadata

#['StorageDT']

#.columns

In [ ]:
from DataLoaders.BodyMeasuresLoader import BodyMeasuresLoader

bml = BodyMeasuresLoader()

bmld = bml.get_data(study_ids=[6,1000])

bmld.df[bmld.df.index.get_level_values(0).isin(t1d_mbl.df_metadata['RegistrationCode'].values)]

In [ ]:
bmld.df.loc[control_d2_metadata['RegistrationCode'].values]['bmi'][bmld.df.loc[control_d2_metadata['RegistrationCode'].values]['bmi']<50].dropna().hist()

plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [5, 3]

bmld.df.loc[t1d_adults_numbers]['bmi'].dropna().hist()

plt.show()

In [ ]:
ttest_ind(bmld.df.loc[control_d2_metadata['RegistrationCode'].values]['bmi'].dropna(),bmld.df.loc[t2d_adults_numbers]['bmi'].dropna())

In [ ]:
from DataLoaders.GutMBLoader import GutMBLoader

mbl = GutMBLoader()

pnp3_mbl = mbl.get_data(df='segata_species', study_ids=17)

pnp3_mbl.df_metadata

In [ ]:
pnp3_mbl.df_metadata[pnp3_mbl.df_metadata['RegistrationCode'].isin(['117111', '724901', '297705', '258667', '279694', '979162', '57724', '922039', '381094',
'78283', '358113', '470856', '393528', '143892', '900460', '41275','745073', '328155', '132095'])].sort_values(['RegistrationCode', 'StorageDT'])

In [ ]:
['117111', '724901', '297705', '258667', '279694', '979162', '57724', '922039', '381094',
'78283', '358113', '470856', '393528', '143892', '900460', '41275','745073', '328155', '132095']


In [ ]:
from DataLoaders.SerumLoader import SerumLoader

In [ ]:
SerumLoader().get_data(study_ids=28).df_metadata

family

In [ ]:
all_mbl = GutMBLoader()
all_mb = all_mbl.get_data(df = 'segata_species', take_log=True, min_col_present_frac=0.2)

In [ ]:
family = pd.read_excel('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D//family.xlsx')

In [ ]:
rel_pathways = pd.read_pickle('/net/mraid08/export/genie/LabData/Data/StudySpecificData/T1D/relative_pathways.df')
all_path =rel_pathways.notna().sum()
interesting_pathways = all_path[all_path>100].index
rel_pathways = interesting_pathways


In [ ]:
all_path =rel_pathways.notna().sum()
interesting_pathways = all_path[all_path>100].index

#.hist()